# 02 文件、进/线程及网络编程

内容导航：

1. I/O编程

2. 进程与线程

3. 网络编程

## 2.1 文件操作

I/O在计算机中是指Input/Output,即输入/输出。只要设计数据交换，都会涉及IO编程，如文件读写、网络数据传输。

### 文件读写

#### 1. 打开文件

文件读写之前需要打开文件，确定文件的读写模式。Python内置了文件打开、读写函数。

先查看open函数的用法：

In [29]:
# help(open)

In [26]:
f = open(r'test01.txt', 'r')

#### 2. 文件模式

open函数中的mode参数用于指定文件的操作模式：

参数值|含义
:--:|:--
'r'| 读模式
'w'| 写模式
'a'| 追加模式
'b'| 二进制模式
'+'| 读和写模式

#### 3. 文件缓冲区

open函数中的第三个可选参数buffering控制着文件的缓冲。
* 如果参数值为0，则不使用缓冲（低性能，不推荐）；
* 如果参数值为1，则使用缓冲，数据写到内存，使用flush函数或者关闭(close)文件时，数据才会写入到硬盘；
* 如果参数值大于1，则该参数代表缓冲区的大小（字节数），-1或者任何负数则代表默认缓冲区大小。

#### 4. 读取文件

文件读取分为按字节读取和按行读取，常用的方法有read()、readlines()：


In [32]:
f = open('test01.txt', 'r')
for line in f:
    print(line)

第一章、爬虫理论基础

第二章、IO、进程线程和网络编程

第三章、第一个爬虫程序

第四章、HTML解析


#### 5. 关闭文件

文件用完之后必须关闭，因为文件是操作系统资源，不及时关闭会影响系统的IO性能。

In [14]:
f.close()

#### 6. 异常处理

外部世界难以预测，任何IO操作都可能会出现异常（IO异常，IOError）,为了保证程序的健壮性，通常需要用try...except...finally进行异常处理。如：

In [18]:
try:
    f = open(r'test02.txt')
    print(f.read())    
except IOError as e:
    print(e)
finally:
    f.close()

[Errno 2] No such file or directory: 'test02.txt'


上面代码略长，Python提供了更为简单的（上下文管理器）语法，使用with语句代替try...finally以及close()方法。如：

In [19]:
with open(r'test01.txt') as f:
    for line in f:
        print(line.strip())

第一章、爬虫理论基础
第二章、IO、进程线程和网络编程
第三章、第一个爬虫程序
第四章、HTML解析


#### 7. 文件写入

写文件和读文件类似，区别是文件模式要使用'w','a'或者'wb':

In [17]:
with open(r'test01.txt', 'a') as f:
    f.write('第四章、HTML解析')

### 序列化操作

序列化就是将内存中的对象编码为可存储或可传输的数据的过程。将内存中的对象序列化之后，可以把序列化后的数据写入磁盘，或者通过网络传输到其他机器，实现程序状态的保存和共享。反过来，从序列化的数据中解码还原出内存对象的过程，称为反序列化。

Python用来序列化/反序列化的模块有：cPickle和pickle。两个模块提供的序列化操作函数是一样的，只是前者性能更好。

pickle实现序列化的操作是dumps()/dump()方法，前者可以将任意对象序列化为字符串(str对象），后者可以将序列化后的对象直接写入到文件中。

示例如下：

In [34]:
!pip show lxml

Name: lxml
Version: 4.3.2
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: http://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: /anaconda3/lib/python3.7/site-packages
Requires: 
Required-by: 


In [36]:
import pickle as pk
d = {'name': 'xiaobai', 'age': 36, 'sex': 'M'}
s = pk.dumps(d) # 序列化为字符串

with open(r'dump.txt', 'wb') as f:
    pk.dump(d, f) # 序列化到文件

In [42]:
s

b'\x80\x03}q\x00(X\x04\x00\x00\x00nameq\x01X\x07\x00\x00\x00xiaobaiq\x02X\x03\x00\x00\x00ageq\x03K$X\x03\x00\x00\x00sexq\x04X\x01\x00\x00\x00Mq\x05u.'

pickle实现反序列化的操作是loads()/load()方法，前者将字符串反序列化为对象，后者将文件反序列化为对象，如下所示：

In [27]:
d1 = pk.loads(s)
d1

{'name': 'xiaobai', 'age': 36, 'sex': 'M'}

In [32]:
with open(r'dump.txt', 'rb') as f:
    d2 = pk.load(f)
d2

# input = read = 反序列化
# output = write = 序列化

{'name': 'xiaobai', 'age': 36, 'sex': 'M'}

如果要在不同系统，不同平台之间传递对象，通常需要把对象序列化为标准格式，如XML。现在更加流行的是序列化为JSON格式。关于JSON，大家可以从[【菜鸟教程】](https://www.runoob.com/)或[【w3school】](https://www.w3school.com.cn/)学习。

## 2.2 进程和线程

在爬虫开发中，进程和线程的概念是非常重要的，提高爬虫的工作效率，打造分布式爬虫，都离不开进程和线程。本节将从多进程、多线程、协程和分布式进程等四个方面学习Python中进程和线程中的常用操作，以便在接下来的爬虫开发中灵活运用进程和线程。

### 多进程

Python实现多进程的方式主要有两种，一种是使用os模块中的fork方法，另一种是使用multiprocessing模块。这两种方法的区别是前者仅适用于Unix/Linux操作系统，Windows不支持，后者则是跨平台的实现方式。

#### 1. 使用os模块的fork函数实现多进程

Python的os模块中封装了常见的系统调用，其中就有fork方法。fork方法来自于Unix/Linux操作系统中提供的一个fork系统调用。例如：

In [37]:
import os

print ('Current Process ({}) start ...'.format(os.getpid()))

pid = os.fork()

if pid < 0:
    print ('Error:fork')
elif pid == 0:
    print ('I am Child Process({0}) and my Parent Process is ({1})'
           .format(os.getpid(),os.getppid()))
else:
    print ('I({0}) Created a Child Process({1}).'.format(os.getpid(), pid))

Current Process (34139) start ...
I(34139) Created a Child Process(40983).
I am Child Process(40983) and my Parent Process is (34139)


#### 2. 使用multiprocessing模块创建多进程
此模块提供了一个Process类来描述一个进程对象。创建子进程时，只需传入一个函数及其参数，即可完成一个Process实例的创建。
使用进程对象的start()方法启动（执行）进程，用join()方法实现进程间的同步。代码如下：

In [29]:
!python ch02_process.py

Parent Process(55462) is creating and     starting Child Processes...
Child Process is starting...
Child Process 0(55463) is running...
Child Process is starting...
Child Process 1(55465) is running...
Child Process is starting...
Child Process 2(55466) is running...
Child Process is starting...
Child Process 3(55467) is running...
Child Process is starting...
Child Process 4(55468) is running...
All Processes Done.


In [25]:
!cat ch02_process.py

import os, time, random
from multiprocessing import Process


# 子进程启动时要执行的函数
def proc_run(name):
    time.sleep(random.random() * 3)
    print('Child Process {0}({1}) is running...'.format(name, os.getpid()))  

print('Parent Process({}) is creating and starting Child Processes...'.format(os.getpid()))

for i in range(5):
    p = Process(target=proc_run(str(i)), args=(str(i), 2))
    print('Child Process is starting...')
    p.start()
    # 等待所有子进程结束
    p.join()
print('All Processes Done.')Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Process objects represent activity that is run in a separate process
 |  
 |  The class is analogous to `threading.Thread`
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseP

以上介绍了创建进程的两种方法，但要启动大量的子进程，使用进程池批量创建子进程的方式更加常见。

#### 3. 使用multiprocessing模块的Pool类创建进程池

Pool可以提供指定数量的进程供用户调用，默认大小为CPU的核数。当有新的任务请求提交到Pool时，如果进程池不满，那么就会创建一个新的进程来执行该任务请求；如果池中的进程数已经达到规定的最大值，那么该请求就会等待，直到池中有进程结束，才会创建（安排）新的进程来处理它。代码如下：

In [30]:
!python ch02_proc_pool.py

Parent Process(55474) is requesting Child Processes for running Task... 
Waiting for all Child Processes in Pool Done.
Task 0 (pid = 55475) is running ...
Task 1 (pid = 55476) is running ...
Task 2 (pid = 55477) is running ...
Task 2 Done.
Task 3 (pid = 55477) is running ...
Task 1 Done.
Task 4 (pid = 55476) is running ...
Task 0 Done.
Task 4 Done.
Task 3 Done.
All Processes Done.


#### 4. 进程间通信

为了使多个进程协作，共同完成复杂任务，进程间的通信是必不可少的。Python提供了多种进程间通信的方式，如Queue（队列）、Pipe（管道）和Value+Array（值+数组）等。这里只介绍Queue和Pipe这两种方式。

Queue和Pipe的区别在于Pipe用来实现两个进程间的通信，Queue则可实现任意多个进程间的通信。

**（1）使用Queue实现多进程之间的数据传递**，有两个方法：put和get，分别用来进行入队和出队操作
* put方法：用以插入一个元素到队列中，即入队。如果对已满，会抛出Queue.Full异常。
* get方法：用以从队列中取出并删除一个元素，即出队。如果对已空，会抛出Queue.Empty异常

下面例子中，父进程创建三个子进程，两个子进程（生产者）往Queue中写入数据，一个子进程（消费者）从Queue中读取数据。代码如下：

In [46]:
!python ch02_proc_queue.py

Producer Process(56833) is putting urls:
Putting http://www.baidu.com to Queue...Done.
Consumer Process(56834) is getting urls:
Putting http://www.taobao.com to Queue...Done.
Putting http://www.126.com to Queue...Done.
Get http://www.baidu.com from Queue.
Putting https://www.sina.com.cn/ to Queue...Done.
Putting http://www.jd.com to Queue...Done.
Putting https://www.ifeng.com/ to Queue...Done.
All Done.
Consumer Process(56835) is getting urls:
Get http://www.taobao.com from Queue.
百度一下，你就知道
Get http://www.126.com from Queue.
126网易免费邮--你的专业电子邮
Get https://www.sina.com.cn/ from Queue.
淘宝网 - 淘！我喜欢
Get http://www.jd.com from Queue.
京东(JD.COM)-正品低价、品质保障、配送及时、轻松购物！
Get https://www.ifeng.com/ from Queue.
凤凰网
新浪首页


**（2）使用Pipe实现两个进程间的通信**，两个进程分别位于管道的两端。

* multiprocessing.Pipe()方法返回管道的两个端。该方法的duplex参数用来指定管道是否为全双工模式。
* 若该参数为True（默认）,两个端均可收发，否则第一个只能收，第二个只能发，收发消息分别使用recv()和send()方法。

下面的例子中，一个子进程通过Pipe发送数据，一个子进程通过Pipe接收数据。代码如下：

In [6]:
!python ch02_proc_pipe.py

Process(58496) send: http://www.000.com
Process(58497) recv: http://www.000.com
Process(58496) send: http://www.111.com
Process(58497) recv: http://www.111.com
Process(58496) send: http://www.222.com
Process(58497) recv: http://www.222.com
Process(58496) send: http://www.333.com
Process(58496) send: http://www.444.com
Process(58497) recv: http://www.333.com
Process(58497) recv: http://www.444.com
Process(58496) send: http://www.555.com
Process(58496) send: http://www.666.com
Process(58497) recv: http://www.555.com
Process(58496) send: http://www.777.com
Process(58496) send: http://www.888.com
Process(58497) recv: http://www.666.com
Process(58496) send: http://www.999.com
Process(58497) recv: http://www.777.com
Process(58497) recv: http://www.888.com
Process(58497) recv: http://www.999.com


### 多线程

多线程就是在同一进程中同时（并发地）执行多个不同的程序。多线程具有如下优点：

* 可以把耗时任务放到后台处理
* 可以提高UI的用户体验，如显示进度条
* 加快程序的运行速度
* 处理高并发任务

Python的标准库提供了两个模块：thread和threading用来实现多线程编程。其中thread为低级模块，threading为高级模块，是对thread的进一步封装，更加好用。这里我们使用threading高级模块。

#### 1. 用threading模块创建多线程

使用threading模块有两种方式创建多线程：

1. 以一个函数为参数创建Thread实例，然后调用该实例的start方法启动线程
2. 通过继承threading.Thread类创建新的线程类，然后重写`__init__`和`run`方法

第一种方法的示例如下：

In [11]:
import random, os
import time, threading

cur_thread_name = threading.current_thread().name
# 新线程执行的函数
def thread_run(urls):
    cur_thread_name = threading.current_thread().name
    print('{0} is running in Process({1})'.format(cur_thread_name, os.getpid()))
    for url in urls:
        print('{0} ---->>> {1}'.format(cur_thread_name, url))
        time.sleep(random.random())
    print('{} Done.'.format(cur_thread_name))

# 主线程
print('{0} is running in Process({1})'.format(cur_thread_name, os.getpid()))
# 创建线程1
urls = [('http://www.' + str(i+1) * 3 + '.com') for i in range(5)]
t1 = threading.Thread(target=thread_run, name="Thread_1", args=(urls,))
# 创建线程2
urls = [('http://www.' + i * 3 + '.org') for i in 'ABCDE']
t2 = threading.Thread(target=thread_run, name="Thread_2", args=(urls,))
# 启动线程
t1.start()
t2.start()
# 同步
t1.join()
t2.join()

print('{} ended.'.format(threading.current_thread().name))


MainThread is running in Process(58159)
Thread_1 is running in Process(58159)
Thread_1 ---->>> http://www.111.com
Thread_2 is running in Process(58159)
Thread_2 ---->>> http://www.AAA.org
Thread_2 ---->>> http://www.BBB.org
Thread_1 ---->>> http://www.222.com
Thread_2 ---->>> http://www.CCC.org
Thread_2 ---->>> http://www.DDD.org
Thread_2 ---->>> http://www.EEE.org
Thread_1 ---->>> http://www.333.com
Thread_1 ---->>> http://www.444.com
Thread_1 ---->>> http://www.555.com
Thread_2 Done.
Thread_1 Done.
MainThread ended.


第二种方法是从threading.Thread继承创建线程类。下面将方法一的程序进行重写，代码如下：

In [12]:
import random as rnd
from threading import Thread
import time

# 自定义线程类
class MyThread(Thread):
    # 构造方法
    def __init__(self, name, urls):
        Thread.__init__(self, name=name)
        self.urls = urls
    # 重写父类的run方法，重写定义线程的行为
    def run(self):
        print('{} is running...'.format(self.name))
        for url in self.urls:
            print('{0} ---->>> {1}'.format(self.name, url))
            time.sleep(random.random())
        print('{} Done.'.format(self.name))

# 主（父）线程
print('{} is running...'.format(threading.current_thread().name))

urls = [('http://www.' + str(i+1) * 3 + '.com') for i in range(5)]
t1 = MyThread(name="Thread_1", urls = urls)
# 创建线程2
urls = [('http://www.' + i * 3 + '.org') for i in 'ABCDE']
t2 = MyThread(name="Thread_2", urls = urls)

# 启动线程
t1.start()
t2.start()
# 同步
t1.join()
t2.join()

print('{} ended.'.format(threading.current_thread().name))

MainThread is running...
Thread_1 is running...
Thread_1 ---->>> http://www.111.com
Thread_2 is running...
Thread_2 ---->>> http://www.AAA.org
Thread_2 ---->>> http://www.BBB.org
Thread_2 ---->>> http://www.CCC.org
Thread_1 ---->>> http://www.222.com
Thread_1 ---->>> http://www.333.com
Thread_2 ---->>> http://www.DDD.org
Thread_2 ---->>> http://www.EEE.org
Thread_1 ---->>> http://www.444.com
Thread_2 Done.
Thread_1 ---->>> http://www.555.com
Thread_1 Done.
MainThread ended.


#### 2. 线程同步

如果多个线程同时对某个数据修改，则可能出现不可预料的结果，为了保证数据的正确性，需要对多个线程进行同步。使用Lock和RLock锁可以实现简单的线程同步，这两个对象都有acquire和release方法，分别用来获取和释放线程锁。

对于Lock对象而言，如果一个线程连续两次对其进行acquire操作，那么由于第一次acquire之后没有release,第二次acquire将挂起线程，导致Lock对象永远不能释放，使得线程死锁。

RLock对象允许一个线程多次对其进行acquire操作，因为其内部有一个计数器counter变量记录着线程acquire的次数，而且每次acquire都有一个release操作与之对应，只有所有的release操作完成之后，别的线程才能申请该RLock对象。

下列代码演示了线程同步的过程：

In [19]:
import threading

my_lock = threading.RLock()

num  = 0

class MyThread(threading.Thread):
    
    def __init__(self, name):
        threading.Thread.__init__(self, name=name)
    
    def run(self):
        global num
        while True:
            my_lock.acquire()
            print('{0} locked, Number: {1}'.format(self.name, num))
            if num >= 4:
                my_lock.release()
                print('{0} released. Number: {1}'.format(self.name, num))
                break
            num += 1
            print('{0} released. Number: {1}'.format(self.name, num))
            my_lock.release()

t1 = MyThread('Thread_1')
t2 = MyThread('Thread_2')

t1.start()
t2.start()


Thread_1 locked, Number: 0
Thread_1 released. Number: 1
Thread_1 locked, Number: 1
Thread_1 released. Number: 2
Thread_1 locked, Number: 2
Thread_1 released. Number: 3
Thread_1 locked, Number: 3
Thread_1 released. Number: 4
Thread_1 locked, Number: 4
Thread_1 released. Number: 4
Thread_2 locked, Number: 4
Thread_2 released. Number: 4


#### 3. 全局解释器锁（GIL）

Python的原始解释器CPython中存在着**GIL（Global Interpreter Lock，全局解释器锁）**，因此在解释执行Python代码时，会产生互斥锁来限制线程对共享资源的访问，直到解释器遇到I/O操作或者操作次数达到一定次数才会释放GIL

由于全局解释器锁的存在，在进行多线程操作的时候，**不能**使用多个CPU内核，只能使用一个内核，所以在进行CPU密集型操作的时候，不推荐使用多线程，更加倾向于多进程。

对于**I/O密集型**操作，多线程可以明显提高效率。例如Python爬虫的开发，绝大多数时间爬虫是在等待socket返回数据，网络IO操作延时比CPU大得多。因此，使用多线程实现爬虫，可以显著地提高爬取效率。

### 协程

**协程（coroutine）**，又称微线程，纤程，是一种用户级的轻量级线程。协程拥有自己的寄存器上下文和栈。

Python通过yield提供了对协程的基本支持，但是不完全，而使用第三方gevent库是更好的选择，gevent提供了比较完善的协程支持。

gevent是一个基于协程的Python网络函数库，使用greenlet在libev事件循环顶部提供了一个有高级别并发性的API。

gevent对协程的支持，本质上是greenlet在实现切换工作。greenlet工作流程如下：
假如进行访问网络的IO操作时，出现阻塞，greenlet就显式切换到另一段没有被阻塞的代码段执行，直到原先的阻塞状况消失以后，再自动切换回原来的代码段继续处理。因此，greenlet是一种合理安排的串行方式

由于IO操作非常耗时，经常使程序处于等待状态，有了gevent为我们自动切换协程，就保证总有greenlet在运行，而不是等待IO，这就是协程一般比多线程效率高的原因。

下面通过一个的例子来演示gevent的使用流程，代码如下：

In [13]:
from gevent import monkey
import gevent
import urllib.request as req

monkey.patch_all()

def run_task(url):
    print('Visit --> {}'.format(url))
    try:
        response = req.urlopen(url)
        data = response.read()
        print('{0} bytes received from {1}'.format(len(data), url))
    except Exception as e:
        print(e)
        
def main():
    urls = ['https://github.com/', 'https://www.python.org/', 'http://www.cnblogs.com/']
    greenlets = [gevent.spawn(run_task, url) for url in urls]
    gevent.joinall(greenlets)
    
main()


Visit --> https://github.com/
Visit --> https://www.python.org/
Visit --> http://www.cnblogs.com/
48164 bytes received from http://www.cnblogs.com/
87783 bytes received from https://github.com/
IncompleteRead(32301 bytes read, 16377 more expected)


以上程序主要用了gevent中的spawn方法和joinall方法。spawn方法可以看做是用来形成协程，joinall方法就是添加这些协程任务，并且启动运行。从运行结果来看，3个网络操作是并发执行的，而且结束顺序不同，但其实只有一个线程。

gevent中还提供了对池的支持。当拥有动态数量的greenlet需要进行并发管理（限制并发数）时，就可以使用池，这在处理大量的网络和IO操作时是非常需要的。

接下来使用gevent中pool对象，对上面的例子进行改写，程序如下：

In [28]:
from gevent import monkey
import urllib.request as req
from gevent.pool import Pool

monkey.patch_all()

def run_task(url):
    print('Visit --> {}'.format(url))
    try:
        response = req.urlopen(url)
        data = response.read()
        print('{0} bytes received from {1}.'.format(len(data), url))
    except Exception as e:
        print(e)
    return 'url:{0} ---> finish'.format(url)

def main():
    pool = Pool(2)
    urls = ['https://github.com/', 'https://www.python.org/', 'http://www.cnblogs.com/']
    results = pool.map(run_task, urls)
    for result in results:
        print(result)

main()

Visit --> https://github.com/
Visit --> https://www.python.org/
48537 bytes received from https://www.python.org/.
Visit --> http://www.cnblogs.com/
47888 bytes received from http://www.cnblogs.com/.
87784 bytes received from https://github.com/.
url:https://github.com/ ---> finish
url:https://www.python.org/ ---> finish
url:http://www.cnblogs.com/ ---> finish


## 2.3 网络编程

既然是做爬虫开发，必然需要了解Python网络编程方面的知识。计算机网络是把各个计算机连接到一起，让网络中的计算机可以互相通信。网络编程就是如何在程序中实现两台计算机的通信。

提到网络编程，必须提到的一个概念是Socket。Socket（套接字）是网络编程的一个抽象概念，通常我们用一个Socket表示“打开了一个网络连接”，而打开一个Socket需要知道目标计算机的IP地址和端口号，再指定协议类型即可。Python提供了两个基本的Socket模块：

* Socket，提供了标准的BSD Sockets API。
* SocketServer，提供了服务器中心类，可以简化网络服务器的开发。

### TCP编程

网络编程一般包括两部分：服务端和客户端。TCP是一种面向连接的通信方式，主动发起连接的叫客户端，被动响应连接的叫服务端。

创建和运行TCP**服务端**一般需要五个步骤：

1. 创建Socket，绑定Socket到本地IP与端口。
2. 开始监听连接。
3. 进入循环，不断接收客户端的连接请求。
4. 接收传来的数据，并发送给对方数据。
5. 传输完毕后，关闭Socket。

下面通过一个例子演示创建TCP服务端的过程，程序如下（须在独立的进程中运行）：

In [1]:
!cat ch02_tcp_server.py

import socket
import threading
import time

def dealClient(sock, addr):
    # 第四步：接收传来的数据，并发送给对方数据
    print('Accept new connection from {0}...'.format(addr))
    sock.send(b'Server: Hello, I am server!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        print('-->>{}'.format(data.decode('utf-8')))
        sock.send(('Server: {}'.format(data.decode('utf-8'))).encode('utf-8'))
    # 第五步：关闭Socket
    sock.close()
    print('Connection from %s:%s closed.' % addr)

def main():
    # 第一步：创建一个基于IPv4和TCP协议的Socket
    # Socket绑定的IP(127.0.0.1为本机IP)与端口
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('127.0.0.1', 9999))
    # 第二步：监听连接
    s.listen(5)
    print('Waiting for connection...')
    while True:
        # 第三步：接收一个新连接：
        sock, addr = s.accept()
        # 创建新线程来处理TCP连接：
        t = threading.Thread(target=dealClient, args=(sock, addr))
        t.start()
if __name_

接着编写**客户端**，与服务端进行交互，TCP客户端的创建和运行需要三个步骤：

1. 创建Socket，连接远端地址。
2. 连接后发送数据和接收数据。
3. 传输完毕后，关闭Socket。

程序如下（须在[新的窗口](ch02_tcp_client.ipynb)运行）：

```
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('127.0.0.1', 9999))
print('-->>'+ s.recv(1024).decode('utf-8'))
s.send(b'Hello, I am a client')
print('-->>'+ s.recv(1024).decode('utf-8'))
s.send(b'exit')
s.close()
```

### UDP编程

TCP通信需要一个建立可靠连接的过程，而且通信双方以流的形式发送数据。相对于TCP, UDP则是面向无连接的协议。使用UDP协议时，不需要建立连接，只需要知道对方的IP地址和端口号，就可以直接发数据包，但是不关心是否能到达目的端。虽然用UDP传输数据不可靠，但是由于它没有建立连接的过程，速度比TCP快得多，对于不要求可靠到达的数据，就可以使用UDP协议。

使用UDP协议，和TCP一样，也有服务端和客户端之分。UDP编程相对于TCP编程比较简单，服务端创建和运行只需要三个步骤：

1. 创建Socket，绑定指定的IP和端口
2. 直接发送数据和接收数据
3. 关闭Socket

服务器端示例程序如下:

客户端示例程序如下，须在[新的窗口](ch02_udp_client.ipynb)运行：

```
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
for data in [b'ABC', b'CDE', b'DEF']:
    s.sendto(data, ('127.0.0.1',9999))
    print(s.recv(1024).decode('utf-8'))
s.close()
```

In [ ]:
!python ch02_udp_server.py

Bind UDP on 9999...
Received from ('127.0.0.1', 64000)
Received from ('127.0.0.1', 64000)
Received from ('127.0.0.1', 64000)
